## (Not finished) Logistic regression in TensorFlow with eager mode

A logistic regression implemented using TensorFlow's Eager API
* Author: Gao Yang

#### Setup eager mode

In [1]:
from __future__ import absolute_import, print_function, division

import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

print('Enabling eager execution...')
tfe.enable_eager_execution()

print('Eager execution status: {}'.format(tf.executing_eagerly()))

/Users/Yang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Enabling eager execution...
Eager execution status: True


#### MNIST dataset
This example uses MNIST dataset. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1.  
For simplicity, each image has streched and converted to an 1-D array of 28x28=784 features.

In [24]:
import os

def load_mnist(file_path):
    assert os.path.isfile(file_path)
    data = np.load(file_path)
    train_x,train_y = data['x_train'],data['y_train']
    test_x,test_y = data['x_test'],data['y_test']
    data.close()
    return (train_x,train_y),(test_x,test_y)

# mnist.npz was stored in local machine
mnist_path = '/Users/Yang/Projects/aymericdamien-tensorflow-examples/notebooks/7_MyCode/mnist.npz'

# load numpy arries from mnist.npz
(train_x_raw,train_y),(test_x_raw,test_y) = load_mnist(mnist_path)

In [28]:
train_x.shape

(60000, 784)

In [26]:
train_x = train_x_raw.reshape(60000, 28*28)

test_x = test_x_raw.reshape(10000, 28*28)


#### Setup the model

In [33]:
num_epoch = 1001
learning_rate = 0.01

W = tfe.Variable(tf.zeros([784,10]), name = 'Weights')
b = tfe.Variable(tf.zeros([10]), name = 'bias')

def model(inputs):
    return tf.matmul(inputs, W) + b

def loss(model, inputs, labels):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = model(inputs), labels = labels))

def accuracy_fn(model, inputs, labels):
    prediction = tf.nn.softmax(model(inputs))
    correct_prediction = tf.equal(tf.argmax(prediction, 1), labels)
    return tf.reduce_mean(tf.cast(correct_prediction, float32))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
grad = tfe.implicit_gradients(loss)